In [24]:
# Make necessary imports
from dotenv import load_dotenv
from openai import OpenAI
from agents import Agent, trace, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Email, To, Content
import os
import asyncio


In [25]:
load_dotenv(override=True)

True

In [20]:
# test sendgrid api by sending a mail
def send_test_email():
    sg_client=SendGridAPIClient(api_key=os.getenv(key="SENDGRID_API_KEY"))
    from_email = Email("sumanprakash.nitp@gmail.com")
    to_email = To("bittu1suman2@gmail.com")
    content= Content("text/plain","This is a test email.")
    mail = Mail(from_email,to_email,"Test Email",content).get()
    response = sg_client.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [22]:
def send_email():
    message = Mail(
    from_email='sumanprakash.nitp@gmail.com',
    to_emails='bittu1suman2@gmail.com',
    subject='Sending with Twilio SendGrid is Fun',
    html_content='<strong>and easy to do anywhere, even with Python</strong>')
    try:
        sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
    # sg.set_sendgrid_data_residency("eu")
    # uncomment the above line if you are sending mail using a regional EU subuser
        response = sg.send(message)
        print(response.status_code)
        print(response.body)
        print(response.headers)
    except Exception as e:
        print(e.message)

send_email()

202
b''
Server: nginx
Date: Thu, 05 Feb 2026 15:33:45 GMT
Content-Length: 0
Connection: close
X-Message-Id: GTB_zP5MQGiLzHbPZ1KMaw
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




In [68]:
# Create 3 System Instructions for 3 different sales agent
instruction1 = f""" You are a ptofessional sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to write professional and serious
cold emails. 
Note: Do not provide subject for the mail
"""
instruction2 = f""" You are a humorous, engaging sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to write witty, engaging
cold emails that are likely to get a response. 
Note: Do not provide subject for the mail
"""
instruction3 = f""" You are a busy sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to concise, to the point cold emails. 
Note: Do not provide subject for the mail 
"""

In [69]:
#Now create 3 different sales agent
sales_agent1= Agent(name="Professional sales agent", instructions=instruction1, model="gpt-4o-mini")
sales_agent2= Agent(name="Humorous sales agent", instructions=instruction2, model="gpt-4o-mini")
sales_agent3= Agent(name="Busy sales agent", instructions=instruction3, model="gpt-4o-mini")


In [70]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am with OOIMA FAASHION, a leading provider of custom apparel and promotional products. We specialize in creating high-quality printed t-shirts, caps, mouse pads, and mugs tailored to meet the unique needs of businesses and organizations like yours.

In today's competitive market, having distinctive branded merchandise can significantly enhance your visibility and foster a deeper connection with your audience. Whether you are looking to promote an event, build brand loyalty, or create memorable giveaways, our wide range of customizable products is designed to help you achieve your goals.

At OOIMA FAASHION, we pride ourselves on our attention to detail, fast turnaround times, and exceptional customer service. We understand that each project is unique, and our team is dedicated to working closely with you to ensure the final product aligns with your vision.

I would love the opportunity to discuss

In [28]:
sales_agents = [sales_agent1, sales_agent2, sales_agent3]

In [34]:
# Run all 3 agents 
message = "write a cold sales mail"
with trace("Parallel Cold Email"):
    results = await asyncio.gather( 
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output+"\n\n")


Subject: Elevate Your Brand with Custom Apparel from OOIMA FAASHION

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent OOIMA FAASHION, a leading provider of custom apparel and promotional products.

In today's competitive market, effectively showcasing your brand is essential. Our high-quality products—ranging from printed t-shirts and caps to custom mugs and mouse pads—are designed to enhance your brand’s visibility and appeal to your target audience.

Here are a few benefits of partnering with us:

1. **Customization:** Tailor products to reflect your brand’s unique identity. 
2. **Quality:** We use premium materials ensuring durability and comfort.
3. **Scalability:** Whether you need a single item or a bulk order, we can accommodate your needs.

I would love the opportunity to discuss how OOIMA FAASHION can help elevate your brand. Are you available for a brief call next week?

Thank you for considering us. I look forward to your 

In [37]:
# Create a new Agent Sales picker whose job is to select and pick the best mail out of the ones written by sales agents
sales_picker = Agent(name="Sales_Picker", 
instructions=f"""You pick the best cold sale emails form the given options. Imagine yourself as a customer 
and pick the one you are highly likely to respond to. Do not give any explanations. 
Just reply with the best cold sale email from the give options.""",
model="gpt-4o-mini")

In [45]:
# Use sales agent to write cold sales email
message = "Please write a cold sale email"
results = await asyncio.gather(
    Runner.run(sales_agent1,message),
    Runner.run(sales_agent2,message),
    Runner.run(sales_agent3,message)
)
outputs = [result.final_output for result in results]
#print(outputs)

# Consolidate the generated cold sale emails
emails = "Cold Sales Emails:\n\n"+"\n\nEmail:\n\n".join(outputs) 
print(emails)
best_email = await Runner.run(sales_picker,emails)
print(f"Best cold sale Email:\n\n {best_email.final_output}")

Cold Sales Emails:

Subject: Elevate Your Brand with Custom Apparel from OOIMA FASHION

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am reaching out on behalf of OOIMA FASHION, a leading provider of high-quality custom apparel and promotional products.

In today’s competitive market, standing out is essential. Our range includes premium printed t-shirts, stylish caps, personalized mouse pads, and custom mugs, all designed to help you promote your brand effectively. Whether you’re looking to enhance your marketing efforts, unite your team with custom gear, or create memorable giveaways, we have the perfect solutions for you.

Here are a few reasons why OOIMA FASHION could be the ideal partner for your branding needs:

1. **Quality Products:** We utilize top-grade materials that ensure longevity and comfort.
2. **Customization Options:** Our design team collaborates closely with you to bring your vision to life, ensuring each product aligns 

In [56]:
#Create a tool to send messages using function tool decorator from openai-agent sdk
@function_tool
def send_email(body: str):
    """Send out an email to all prospects with the given body"""
    sg = SendGridAPIClient(os.getenv("SENDGRID_API_KEY"))
    message = Mail(from_email="sumanprakash.nitp@gmail.com",
    to_emails="bittu1suman2@gmail.com",
    subject="Sales Email",
    plain_text_content=body)
    response = sg.send(message)
    print("Success: Email Sent.")
    return response.status_code

In [50]:
send_email

FunctionTool(name='send_email', description='Send out an email to all prospects with the given body', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AA12E6B100>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [ ]:
#Convcert other sales agent as a tool
tool_description="Write a cold sales email"
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=tool_description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=tool_description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=tool_description)

In [52]:
from agents import tool


tools = [tool1, tool2, tool3, send_email]

In [53]:
# Create a sales manager agent
instructions= """
You are a Sales Manager at OOIMA FASHION. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
    """
sales_manager = Agent(name="Sales_Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

In [55]:
message = "Write a cold sales email addressed to 'Dear CEO'"
with trace("Sales Manager"):
   result = await Runner.run(sales_manager,message)

In [57]:
# Create an agent for writing subjects for a cold sales email asnd then convert it into a tool
subject_instruction = """You can write subject for cold sales email. You will be provided a message and based on that you need
to write a catchy subject that is likely to get customers engagement."""
subject_writer = Agent(name="Subject_Writing_Agent", instructions=subject_instruction, model="gpt-4o-mini")
subject_writing_tool = subject_writer.as_tool(tool_name="Subject_Writing_Tool", tool_description="Write a catchy subject for an email")

In [74]:
reply = await Runner.run(sales_agent1, "Write a cold sales email")
email = reply.final_output
print(email)

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent OOIMA FAASHION, a leader in custom apparel and promotional products. We specialize in creating high-quality printed items such as t-shirts, caps, mouse pads, and mugs that can be tailored to meet your unique branding needs.

At OOIMA FAASHION, we understand the importance of quality and creativity in promotional merchandise. Our products not only serve as excellent marketing tools but also as memorable gifts that leave a lasting impression on your clients and employees. 

Whether you’re looking to boost brand awareness or celebrate a special occasion, our team is dedicated to providing you with exceptional service and products that align with your vision. We offer competitive pricing, quick turnaround times, and a commitment to customer satisfaction.

I would appreciate the opportunity to discuss how we can assist you in enhancing your brand through our customizable products. Please le

In [75]:
sub1 = await Runner.run(subject_writer, email)
print(sub1.final_output)


Subject: Elevate Your Brand with Custom Apparel & Unique Promotional Products!


In [76]:
# Create an agent to convert the mail body from plain text to HTML content
html_instructions = """
You can convert a text email body to HTML email body. You are given text email body which might have some markdown.
Your job is to convert it to an HTML body with simple, clear, compelling layout and design.
"""
html_converter = Agent(name="HTML_Converter_Agent", instructions=html_instructions, model= 'gpt-4o-mini')
html_conversion_tool = html_converter.as_tool(tool_name="html_conversion_tool", tool_description="Convert an email body from plain text to HTML format")

In [78]:
reply = await Runner.run(html_converter, email)
print(reply.final_output)

Here's the HTML email body based on your text:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Email</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            padding: 20px;
            background-color: #f4f4f4;
            color: #333;
        }
        .container {
            background-color: #ffffff;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
        }
        h1 {
            color: #333;
        }
        p {
            margin: 10px 0;
        }
        .signature {
            margin-top: 20px;
            font-size: 14px;
        }
        a {
            color: #007BFF;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>

<div class="container

In [79]:
# Write a tool to send HTML email
@function_tool
def send_html_email(subject: str, html_body: str) -> dict[str, str]:
    """Send out an email with given subject and html body to all sales prospect"""
    sg = SendGridAPIClient(os.getenv("SENDGRID_API_KEY"))
    message = Mail(
        from_email="sumanprakash.nitp@gmail.com",
        to_emails="bittu1suman2@gmail.com",
        subject=subject,
        html_content=html_body
    )
    sg.send(message)
    return {"Status":"Email Sent"}

In [80]:
send_html_email

FunctionTool(name='send_html_email', description='Send out an email with given subject and html body to all sales prospect', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_html_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AA158D16C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [81]:
email_formatting_tools= [subject_writing_tool, html_conversion_tool, send_html_email]

In [83]:
# Emailer agent that is going to work as Handsoff
emailer_instruction = """
You are an email formatter and sender. You recieve the body of the email to be sent. 
You first use the subject_writing_tool to get the subject for the email.
Then, you use the html_conversion_tool to convert the email body in html format.
Finally, You use the send_html_email tool to send the html email.
"""

emailer_agent = Agent(name="Emailer_Agent", 
instructions=emailer_instruction, 
model="gpt-4o-mini", 
tools=email_formatting_tools,
handoff_description="send an email with catchy subject and HTML body")

In [84]:
mail_drafting_tools = [tool1, tool2, tool3]
handoffs =[emailer_agent]

print(mail_drafting_tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AA1579D8A0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AA1579DD00>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', descrip

In [85]:
# Create a updated sales manager with these tools and handoff and appropriate instructions
sales_manager_instructions = """
You are a sales manager at OOIMA FASHION. You have access to mail_drafting_tools to draft mails.
Your job is to act following these instructions below:
1. Use tools in mail_drafting_tools to draft cold sales mails for sending to prospect customers. 
And, always use the tools for drafting emails. Never do it by yourself.
2. Read the drafted mails and pick the best one that is highly likely to get the customer engagement and their response. 
If you are not satisfied, rerun the tools in mail_drafting_tools to get another draft. 
You can rerun the mail_drafting_tools until you are satisfied and get the mail 
that is highly likely to receive the customer response and engagement. 
3. Handoff the best mail to emailer agent who will take care of the email subject,
converting the body in HTML format, and sending the best mail to the customer.
Remember to handoff only and only the best mail to the emailer agent.
"""
sales_manager_agent = Agent(name="Sales_Manager", 
tools=mail_drafting_tools, 
instructions=sales_manager_instructions,
handoffs=handoffs,
model="gpt-4o-mini"
)

In [86]:
message = "send a cold sales email addressed to 'Dear CEO' from Suman."
with trace("Automated SDR"):
    response = await Runner.run(sales_manager_agent, message)